In [ ]:
import os
import json
import regex
import random
from pathlib import Path
from statistics import mean
from huggingface_hub import InferenceClient

### CONFIG

In [ ]:
CONFIG_PATH = Path("Config/huggingface.json")
PROMPT_REASON_PATH = Path("Prompt/Reasoning.txt")
PROMPT_CRITIC_PATH = Path("Prompt/Critical.txt")
TESTDATA_PATH = Path("Data/TestData.json")

with open(CONFIG_PATH, "r", encoding="utf-8") as f:
    config = json.load(f)

HF_TOKEN = config.get("HF_TOKEN")
REASON_MODEL = config.get("REASON_MODEL")
CRITICAL_MODEL = config.get("CRITICAL_MODEL")

os.environ["HF_TOKEN"] = HF_TOKEN
reason_client = InferenceClient(api_key=HF_TOKEN)
critic_client = InferenceClient(api_key=HF_TOKEN)

### PROMPT

In [ ]:
PROMPT_REASON_BASE = """
    Bạn là hệ thống tóm tắt văn bản thông minh.
    Hãy tạo bản tóm tắt chính xác, ngắn gọn, mạch lạc và có thể giải thích.
    1. Phân tích nội dung chính, thực thể, hành động.
    2. Lọc ý quan trọng.
    3. Viết tóm tắt ≤ 100 từ.

    Đầu ra:
    [Reasoning trace]: Chủ đề, Các ý chính, Các ý loại bỏ.
    [Văn bản tóm tắt]: đoạn tóm tắt cuối cùng.

    Văn bản gốc:
    {input_text}
    """

PROMPT_CRITIC_BASE = """
    Bạn là chuyên gia đánh giá reasoning.
    Hãy đọc kỹ văn bản gốc và reasoning, sau đó đánh giá theo 6 tiêu chí.
    Trả về JSON:
    {
    "factuality": <1–5>,
    "clarity": <1–5>,
    "logical_coherence": <1–5>,
    "coverage": <1–5>,
    "utility": <1–5>,
    "consistency": <1–5>,
    "nhan_xet_chung": "...",
    "goi_y_chinh_sua": "..."
    }
    """

In [ ]:
def load_prompt(path: Path, default_text: str) -> str:
    if path.exists():
        with open(path, "r", encoding="utf-8") as f:
            return f.read().strip()
    else:
        print("DEFAUT !!")
        return default_text
    
REASON_PROMPT = load_prompt(
    PROMPT_REASON_PATH,
    PROMPT_REASON_BASE,
)

CRITIC_PROMPT = load_prompt(
    PROMPT_CRITIC_PATH,
    PROMPT_CRITIC_BASE,
)

### HELPERS

In [ ]:
def jsonParse(text: str):
    text = text.strip()
    try:
        return json.loads(text)
    except Exception:
        match = regex.search(r"\{(?:[^{}]|(?R))*\}", text, flags=regex.DOTALL)
        if match:
            json_str = match.group(0)
            try:
                return json.loads(json_str)
            except json.JSONDecodeError:
                pass
        raise ValueError(f"❌ Không parse được JSON từ phản hồi:\n{text[:400]}...")

In [ ]:
def randomContent() -> str:
    if not TESTDATA_PATH.exists():
        raise FileNotFoundError("❌ Thiếu file Data/TestData.json")
    with open(TESTDATA_PATH, "r", encoding="utf-8") as f:
        data = json.load(f)
    items = [x for x in data if "content" in x]
    if not items:
        raise ValueError("❌ Không tìm thấy trường 'content' trong TestData.json")
    sample = random.choice(items)
    print(f"📘 Chọn ngẫu nhiên mục: {sample.get('title', 'Không tiêu đề')}")
    return sample["content"]

In [ ]:
def averageScore(critical_output: dict) -> float:
    keys = ["factuality", "clarity", "logical_coherence", "coverage", "utility", "consistency"]
    vals = [critical_output[k] for k in keys if isinstance(critical_output.get(k), (int, float))]
    return mean(vals) if vals else 0.0

### REASONING

In [ ]:
def reasoningRun(text: str) -> str:
    prompt = REASON_PROMPT.format(input_text=text)
    completion = reason_client.text_generation(
        model=REASON_MODEL,
        prompt=prompt,
        max_new_tokens=1024,
        temperature=0.3,
        top_p=0.9,
        details=True,
    )

    if isinstance(completion, dict) and "generated_text" in completion:
        return completion["generated_text"].strip()
    elif isinstance(completion, list):
        return "\n".join([c.get("generated_text", "") for c in completion]).strip()
    else:
        return str(completion).strip()

### CRITICAL

In [ ]:

def criticalRun(source_text: str, reasoning_output: str) -> dict:
    prompt = (
        f"{CRITIC_PROMPT}\n\n"
        f"[Văn bản gốc]\n{source_text}\n\n"
        f"[Reasoning + Summary]\n{reasoning_output}\n\n"
        "Chỉ trả JSON duy nhất."
    )

    completion = critic_client.text_generation(
        model=CRITICAL_MODEL,
        prompt=prompt,
        max_new_tokens=1024,
        temperature=0.2,
        top_p=0.9,
        details=True,
    )

    # Lấy toàn bộ output từ mô hình
    if isinstance(completion, dict) and "generated_text" in completion:
        raw_output = completion["generated_text"]
    elif isinstance(completion, list):
        raw_output = "\n".join([c.get("generated_text", "") for c in completion])
    else:
        raw_output = str(completion)

    return jsonParse(raw_output)

### MAIN FLOW

In [ ]:
def mainFlow(source_text: str, max_iters=3, min_improve=0.05):
    best_reason, best_score = None, 0
    history = []

    for step in range(1, max_iters + 1):
        print(f"\n🔄 Vòng {step} ...")
        reasoning_output = reasoningRun(source_text)
        critical_output = criticalRun(source_text, reasoning_output)
        average_score = averageScore(critical_output)

        print(f"📊 Điểm TBC: {average_score:.2f}")
        print(f"📝 Nhận xét: {critical_output}\n")

        history.append({"round": step, "score": average_score, "reasoning": reasoning_output, "evaluation": critical_output})
        if average_score > best_score + min_improve:
            best_reason, best_score = reasoning_output, average_score
        else:
            print("⛔ Không cải thiện đáng kể, dừng.")
            break

    return {"best_reasoning": best_reason, "best_score": best_score, "history": history}

### RUN

In [ ]:
if __name__ == "__main__":
    inputPara = randomContent()
    result = mainFlow(inputPara)

    print("\n🎯 Kết quả cuối cùng:")
    print(json.dumps({
        "best_score": result["best_score"],
        "best_reasoning": result["best_reasoning"][:400] if result["best_reasoning"] else None
    }, ensure_ascii=False, indent=2))
